In [1]:
import pandas as pd
import numpy as np
import operator
import warnings
warnings.filterwarnings('ignore')

# Part 1 : Content based filtering using features of cars

In [2]:
df =  pd.read_csv('cars_features.csv', index_col = 0)
df.head()

,Comfort,Performance,Safety,Fuel efficiency,After Sale Support,Styling,Space,Value for money,Avg life of car,Engine,Power
car_id,,,,,,,,,,,
1,1,0,1,0,1,0,1,1,0,0,0
2,0,1,1,0,0,0,0,1,0,1,1
3,1,1,0,1,0,1,0,1,0,0,0
4,1,0,1,0,0,1,0,0,0,1,1
5,0,1,1,0,1,0,0,0,1,1,0


In [3]:
df.drop("Power",  axis = 1, inplace = True)
df.head()

,Comfort,Performance,Safety,Fuel efficiency,After Sale Support,Styling,Space,Value for money,Avg life of car,Engine
car_id,,,,,,,,,,
1,1,0,1,0,1,0,1,1,0,0
2,0,1,1,0,0,0,0,1,0,1
3,1,1,0,1,0,1,0,1,0,0
4,1,0,1,0,0,1,0,0,0,1
5,0,1,1,0,1,0,0,0,1,1


In [4]:
for i in range(1,len(df)):
    print(df.loc[i])
    print("========================")

Comfort               1
Performance           0
Safety                1
Fuel efficiency       0
After Sale Support    1
Styling               0
Space                 1
Value for money       1
Avg life of car       0
Engine                0
Name: 1, dtype: int64
Comfort               0
Performance           1
Safety                1
Fuel efficiency       0
After Sale Support    0
Styling               0
Space                 0
Value for money       1
Avg life of car       0
Engine                1
Name: 2, dtype: int64
Comfort               1
Performance           1
Safety                0
Fuel efficiency       1
After Sale Support    0
Styling               1
Space                 0
Value for money       1
Avg life of car       0
Engine                0
Name: 3, dtype: int64
Comfort               1
Performance           0
Safety                1
Fuel efficiency       0
After Sale Support    0
Styling               1
Space                 0
Value for money       0
Avg life of car       

In [5]:
def Recommend_Similars_Cars(features_vectos, features_priorities):
    
    # user desired features input
    user_desired_features = features_vectos
    # user features priority input
    user_features_priorities = features_priorities
    
    #list to hold the similarity score of cars matching users choice
    similarity_score = []

    # function to calculate similarity scores
    def similar_cars(user_DV):
        user_vector = np.array(user_desired_features)

        for i in range(1,len(df)+1):
            v = list(df.loc[i])
            vector = np.array(v)
            dist = np.linalg.norm(vector - user_vector) 
            similarity_score.append(round(dist, 2))

        return similarity_score
    
    #function to make seprate dataframe for user for further comptations and to find similar cars ids
    def seprate_df():
        
        df["E_distance"] = similar_cars(features_vectos)
        
        #sorting cars score-wise 
        new_df = df.nsmallest(10,'E_distance')

        # new_df = df.drop(df.columns.difference(['E_distance']))
        new_df_temp =  new_df.loc[:, df.columns.intersection(['E_distance'])]

        similar_cars_id = list(new_df_temp.index)
        return similar_cars_id

    
    id_score_dict = {}
    def make_dict(user_features_priorities):
        similar_cars_ids = seprate_df()
        
        # rolling back df to its original form for computation of final similar car score
        df.drop('E_distance', axis = 1, inplace = True)

        for i in similar_cars_ids:
            score = np.dot(user_features_priorities, list(df.loc[i]))
            id_score_dict[i] = score

        return id_score_dict
    
    
    temp_dict = dict(id_score_dict)
    
    # final list that will be containing similar cars ids
    car_id_recommendations = []
    
    def produce_ids():
        
        score_dict = make_dict(user_features_priorities)
        sorted_scores = sorted(score_dict.values(), reverse = True) 

        def get_key(val):
            for key, value in score_dict.items():
                 if val == value:
                     return key

        for i in sorted_scores:
            car_id = get_key(i)
            car_id_recommendations.append(car_id)
            del score_dict[car_id]

        return car_id_recommendations
    
    produce_ids()
    
    print("Top Five recommendation on the basis of users desired features : ")

    for i in range(5):
        print("Recommendation", i+1, " : id =", car_id_recommendations[i])
    

In [6]:
User_DesiredVector = [1, 1, 1, 1, 0, 0, 0, 1, 0, 0]
User_PriorityVector = [2.5, 1, 1.5, 2, 0, 0, 0, 0.5, 0, 0]

Recommend_Similars_Cars(User_DesiredVector, User_PriorityVector)

Top Five recommendation on the basis of users desired features : 
Recommendation 1  : id = 30
Recommendation 2  : id = 3
Recommendation 3  : id = 20
Recommendation 4  : id = 6
Recommendation 5  : id = 14
